In [1]:
import os
import glob
import random
from datetime import datetime

import cv2
import pandas as pd
import numpy as np

import timm
import torch
import torch.nn as nn

import albumentations as A
from albumentations.pytorch import ToTensorV2


SEED = 1111
BATCH_SIZE  = 16
NUM_WORKERS =  4

SEEDS = [5, 111, 1111]

MODEL_NAME = 'nfnet_l2'

random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

cv2.setNumThreads(2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('DEVICE =', device)

df = pd.read_csv('test.csv')
print(df.shape)
df.head(3)

DEVICE = cuda
(2138, 2)


,ID_img,class
0,0.jpg,0
1,1.jpg,0
2,2.jpg,0


In [2]:
class Model(nn.Module):
    def __init__(self, 
                 img_model,
                 classes: int,
                 in_features: int):
        
        super().__init__()
        self.in_features = in_features
        self.img_model = img_model

    def forward(self, x):
        return self.img_model(x)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame,
                 transforms: A.Compose = None,
                 mode: str = 'train'):

        self.mode = mode
        self.cls  = dataframe['class'].values
        self.imgs = dataframe['ID_img'].apply(lambda x: os.path.join(mode, f'{x}')).values
        self.transforms = transforms

    def __getitem__(self, idx) -> dict:

        image_path = self.imgs[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        item = {'image': image}
        if self.transforms is not None:
            item = self.transforms(**item)
        
        item['trg_cls'] = torch.tensor(self.cls[idx]).long()

        return item

    def __len__(self) -> int:
        return len(self.imgs)

In [3]:
img_model = timm.create_model('eca_nfnet_l2', pretrained=False)
in_features = img_model.head.fc.in_features
img_model.head.fc = nn.Linear(in_features, 8)

transform_val = A.Compose([
    A.LongestMaxSize(160, interpolation=cv2.INTER_AREA),
    A.PadIfNeeded(160, 160),
    A.Normalize(mean=img_model.pretrained_cfg['mean'],
                std =img_model.pretrained_cfg['std']),
    ToTensorV2()
])

model = Model(img_model,
              classes = df['class'].nunique(),
              in_features=in_features).cuda()

dataset_test = Dataset(df, transforms=transform_val, mode='test')

loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS, shuffle=False,
                                           drop_last=False, pin_memory=True)

In [4]:
%%time

test_out = None
for seed in SEEDS:
    model.load_state_dict(torch.load(f"{MODEL_NAME}-{seed:04d}.pth", map_location=device))
    model.eval()
    pred = []
    with torch.no_grad():
        for batch in loader_test:
            batch = {k:v.cuda() for k, v in batch.items()}
            pred.append(model(batch['image']).sigmoid().cpu().numpy())
    if test_out is None:
        test_out  = np.vstack(pred)
    else:
        test_out += np.vstack(pred)

CPU times: user 51.6 s, sys: 2.34 s, total: 53.9 s
Wall time: 53.1 s


In [5]:
df['class'] = np.argmax(test_out, axis=1)
df.to_csv(f'{MODEL_NAME}.csv', index=False)
df.head()

,ID_img,class
0,0.jpg,5
1,1.jpg,5
2,2.jpg,1
3,3.jpg,1
4,4.jpg,6
